In [ ]:
# Mout Google Drive
# https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228
from google.colab import drive
ROOT = "/content/drive"
drive.mount(ROOT)
# %pwd %ls
# run github settings
%run /content/drive/MyDrive/CNNStanford/pytorch/pytorch_sandbox/Colab_Helper.ipynb

In [ ]:
MESSAGE = "clean file & gitignore again"
!git config --global user.email "ronyginosar@mail.huji.ac.il"
!git config --global user.name "ronyginosar"
!git add .

In [ ]:
!git commit -m "{MESSAGE}"
!git push "{GIT_PATH}"

Empty

In [ ]:
import torch
import torchvision
import torchmetrics
from torchvision import transforms
from torch.utils.data import DataLoader
import sys
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from tqdm import tqdm

VOC_classes = ['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
               'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
               'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

def main(argv):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Define the model (see https://pytorch.org/vision/stable/models.html#semantic-segmentation)
    # Dont forget to use the pretrained weights (they are downloaded automatically)
    model = ...
    model.eval()
    model = model.to(device)

    # Define transform for image
    # (A) Convert to tensor
    # (B) Normalize with mean & std (see https://pytorch.org/vision/stable/models.html#semantic-segmentation)
    transform = transforms.Compose([
        ...
    ])

    # Define transform for target
    # (A) Convert to tensor
    # (B) Squeeze to 2D label map (HxW)
    # (C) convert float labels to integers: multiply by 255 and cast to long int.
    # (D) Replace void labels (255) with 21 (i.e. define it as the 22'th class, to be ignored later)
    # Hint - use transforms.Lambda for generic transforms
    transform_target = transforms.Compose([
        ...
    ])
    # Define the 'VOC 2012 Val' dataset (see  use the pretrained weights (they are downloaded automatically)
    # Use this root dir: "/stage/algo-datasets/DB/PyTorch/VOC"
    dataset = ...

    #The dataset GT labels have the folloowing form:
    #  0: background
    #  [1.. 20]  interval: segmented objects, classes[Aeroplane, ..., Tvmonitor]
    #  255: void category, used for border regions (5px) and to mask difficult objects


    dataloader = ...

    # Define metrics as torchmetrics modules
    # (documentation: https://torchmetrics.readthedocs.io/en/v0.6.1/references/modules.html#classification-metrics)
    # hints: Use 22 classes and define ignore_index=21
    mIOU = ...
    acc_global = ...
    acc_per_class = ...

    # Test Loop
    with torch.no_grad():
        for i, (images, targets) in enumerate(tqdm(dataloader)):
            images = images.to(device)
            targets = targets.to(device)
            ...
            ...

    # Print results:
    print(f'mIOU = {... :.2f}%')
    print(f'global acc = {... :.2f}%')
    for i, class_name in enumerate(VOC_classes):
        print(f'"{class_name}" accuracy = {... :.2f}%')



if __name__ == "__main__":
    main(sys.argv[1:])

Solution

In [ ]:
import torch
import torchvision
import torchmetrics
from torchvision import transforms
from torch.utils.data import DataLoader
import sys
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from tqdm import tqdm

VOC_classes = ['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
               'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
               'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

def main(argv):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torchvision.models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=True)
    model.eval()
    model = model.to(device)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    transform_target = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.squeeze()),
        transforms.Lambda(lambda x: (255*x).long()),
        transforms.Lambda(lambda x: torch.where(x==255,21,x))
    ])
    dataset = torchvision.datasets.VOCSegmentation(root="__",
                                                   year="2012", image_set="val", download=False,
                                                   transform=transform,
                                                   target_transform=transform_target
                                                   )
    '''
    0: background
    [1.. 20]  interval: segmented objects, classes[Aeroplane, ..., Tvmonitor]    
    255: void category, used for border regions (5px) and to mask difficult objects
    '''

    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    mIOU = torchmetrics.IoU(num_classes=22, ignore_index=21).to(device)
    acc_global = torchmetrics.Accuracy(num_classes=22, average='micro',mdmc_average='global', ignore_index=21).to(device)
    acc_per_class = torchmetrics.Accuracy(num_classes=22, average='none', mdmc_average='global', ignore_index=21).to(device)

    # Test Loop
    with torch.no_grad():
        for i, (images, targets) in enumerate(tqdm(dataloader)):
            images = images.to(device)
            targets = targets.to(device)

            preds = model(images)['out']
            pred_cls = preds.argmax(dim=1)

            mIOU.update(pred_cls,targets)
            acc_global.update(pred_cls,targets)
            acc_per_class.update(pred_cls,targets)

    print(f'mIOU = {100*mIOU.compute():.2f}%')
    print(f'global acc = {100*acc_global.compute():.2f}%')
    acc_per_class = acc_per_class.compute()
    for i, class_name in enumerate(VOC_classes):
        print(f'"{class_name}" accuracy = {100 * acc_per_class[i].item() :.2f}%')


if __name__ == "__main__":
    main(sys.argv[1:])